In [3]:
import pandas as pd
df =pd.read_csv('insurance_pre.csv')

In [4]:
df

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [5]:
df = pd.get_dummies(df,drop_first=True)
df

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
x_independent = df[['age', 'bmi', 'children',  'sex_male', 'smoker_yes']]
y_depenedent = df[['charges']]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_independent,y_depenedent,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_s = sc.fit_transform(x_train) 
x_test_s = sc.transform(x_test)

In [10]:
# create function
#Linear Regression checking
def Linear_Regrssion_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score
    LR_score_df=pd.DataFrame(columns=['Std/normal','r_score'])
    LR_reg = LinearRegression()
    form = ['Standard','Normal']
    for f in form:
        if(f=='Standard'):
            LR_reg.fit(x_train_s,y_train)
            y_pred = LR_reg.predict(x_test_s)
        else:
            LR_reg.fit(x_train,y_train)
            y_pred = LR_reg.predict(x_test)
        LR_score=r2_score(y_test,y_pred)
        LR_score_df.loc[len(LR_score_df.index)] = [ f,LR_score]
    max_score= LR_score_df[LR_score_df['r_score']==LR_score_df['r_score'].max()]
    return (LR_score_df, max_score)

#SVM checking 
def SVM_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    from sklearn.svm import SVR
    from sklearn.metrics import r2_score
    SVM_score_df=pd.DataFrame(columns=['Std/normal','Kernel','C','r_score'])
    form=['Standard','Normal']
    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    C=[1,10,100,1000,10000,100000]
    for f in form:
        for i in kernel:
            for j in C:
                svm_reg = SVR(kernel=i,C=j)
                if(f=='Standard'):
                    svm_reg.fit(x_train_s,y_train)
                    y_pred = svm_reg.predict(x_test_s)
                else:
                    svm_reg.fit(x_train,y_train)
                    y_pred = svm_reg.predict(x_test)
                svm_r2score=r2_score(y_test,y_pred)
                SVM_score_df.loc[len(SVM_score_df.index)] = [ f ,i , j ,svm_r2score]
    max_score= SVM_score_df[SVM_score_df['r_score']==SVM_score_df['r_score'].max()]
    return (SVM_score_df,max_score)

# checking Decision tree
def DT_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.metrics import r2_score
    DT_score_df=pd.DataFrame(columns=['Std/Normal','criterion','splitter','r_score'])
    criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
    splitter =['best', 'random']
    form=['Standard','Normal']    
    for f in form:
        for i in criterion:
            for j in splitter:
                DT_reg=DecisionTreeRegressor(criterion=i,splitter=j)
                if (f=='Standard'):
                    DT_reg.fit(x_train_s,y_train)
                    y_pred = DT_reg.predict(x_test_s)
                else:
                    DT_reg.fit(x_train,y_train)
                    y_pred = DT_reg.predict(x_test)
                DT_r_score = r2_score(y_test,y_pred)
                DT_score_df.loc[len(DT_score_df.index)] = [ f,i , j ,DT_r_score]
    max_score= DT_score_df[DT_score_df['r_score']==DT_score_df['r_score'].max()]
    return (DT_score_df,max_score)
 
#random forest checking
def RF_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import r2_score
    RF_score_df=pd.DataFrame(columns=['std/normal','n_estimators','r_score'])
    n_estimators = [10,50,100,150,200]
    form=['Standard','Normal']
    for f in form:
        for i in n_estimators:
            RF_reg=RandomForestRegressor(n_estimators=i,random_state =0)
            if(f=='Standard'):
                RF_reg.fit(x_train_s,y_train)
                y_pred = RF_reg.predict(x_test_s)
            else:
                RF_reg.fit(x_train,y_train)
                y_pred = RF_reg.predict(x_test)
            RF_r_score = r2_score(y_test,y_pred)
            RF_score_df.loc[len(RF_score_df.index)] = [f, i , RF_r_score]
    max_score= RF_score_df[RF_score_df['r_score']==RF_score_df['r_score'].max()]
    return (RF_score_df,max_score)

#adaboosting checking 
def AdaBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    from sklearn.ensemble import AdaBoostRegressor
    from sklearn.metrics import r2_score
    AdaBoost_score_df=pd.DataFrame(columns=['std/normal','n_estimators','r_score'])
    n_estimators = [10,50,100,150,200]
    form=['Standard','Normal']
    for f in form:
        for i in n_estimators:
            AdaBoost_reg=AdaBoostRegressor(n_estimators=i,random_state =0)
            if(f=='Standard'):
                AdaBoost_reg.fit(x_train_s,y_train)
                y_pred = AdaBoost_reg.predict(x_test_s)
            else:
                AdaBoost_reg.fit(x_train,y_train)
                y_pred = AdaBoost_reg.predict(x_test)
            AdaBoost_r_score = r2_score(y_test,y_pred)
            AdaBoost_score_df.loc[len(AdaBoost_score_df.index)] = [f, i , AdaBoost_r_score]
    max_score= AdaBoost_score_df[AdaBoost_score_df['r_score']==AdaBoost_score_df['r_score'].max()]
    return (AdaBoost_score_df,max_score)   
            

In [11]:
LR_all,LR_max = Linear_Regrssion_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)
SVM_all,SVM_max=SVM_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)
DT_all,DT_max=DT_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)
RF_all,RF_max=RF_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)
AdaBoost_all,AdaBoost_max,AdaBoost_reg=AdaBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
LR_max

,Std/normal,r_score
0,Standard,0.789479
1,Normal,0.789479


In [25]:
all_model_result = pd.DataFrame(columns=['model_name','hyperparamter_value','r2score'] )
all_model_result.loc[len(all_model_result.index)] = ['Multiple_LinearRegression','Normal','0.789479']

In [26]:
all_model_result

,model_name,hyperparamter_value,r2score
0,Multiple_LinearRegression,Normal,0.789479


In [27]:
SVM_max

,Std/normal,Kernel,C,r_score
2,Standard,rbf,100000,0.872498


In [28]:
all_model_result.loc[len(all_model_result.index)] = ['SVM','form=Standard,Kernel = rbf,c=100000','0.789479']

In [30]:
all_model_result

,model_name,hyperparamter_value,r2score
0,Multiple_LinearRegression,Normal,0.789479
1,SVM,"form=Standard,Kernel = rbf,c=100000",0.789479


In [31]:
DT_max

,Std/Normal,criterion,splitter,r_score
15,Normal,poisson,random,0.759623


In [32]:
all_model_result.loc[len(all_model_result.index)] = ['Decision tree','form=normal,criterion =poisson splitter=random','0.759623']

In [33]:
all_model_result

,model_name,hyperparamter_value,r2score
0,Multiple_LinearRegression,Normal,0.789479
1,SVM,"form=Standard,Kernel = rbf,c=100000",0.789479
2,Decision tree,"form=normal,criterion =poisson splitter=random",0.759623


In [34]:
RF_max

,std/normal,n_estimators,r_score
0,Standard,200,0.853603


In [35]:
all_model_result.loc[len(all_model_result.index)] = ['Random Forest','form=Standard,n_estimators =200','0.853603']

In [36]:
all_model_result

,model_name,hyperparamter_value,r2score
0,Multiple_LinearRegression,Normal,0.789479
1,SVM,"form=Standard,Kernel = rbf,c=100000",0.789479
2,Decision tree,"form=normal,criterion =poisson splitter=random",0.759623
3,Random Forest,"form=Standard,n_estimators =200",0.853603


In [38]:
all_model_result[all_model_result['r2score']==all_model_result['r2score'].max()]

,model_name,hyperparamter_value,r2score
3,Random Forest,"form=Standard,n_estimators =200",0.853603


In [40]:
RF_all,RF_max=RF_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,),

In [41]:
import pickle 
pickle.dump(RF_reg,open("insurance_premium_RF.sav",'wb'))

In [42]:
loaded_model= pickle.load(open("insurance_premium_RF.sav",'rb'))

In [46]:
result = loaded_model.predict([[19,27.900,0,0,1]])

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [47]:
result

array([17016.8502575])